# Matrix Factorization

In [3]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
                 learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 0):
        self.Y_raw = Y_data.copy()
        self.Y_data = Y_data.copy()
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
        
        # item biases
        self.b = np.random.randn(self.n_items)
        self.d = np.random.randn(self.n_users)
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
#         self.mu = np.mean(Y_data[:, 2])
        self.mu = 0
 

    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_data[:, user_col] 
        self.muu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 
#             print m
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.muu[n] = m
            # normalize
            self.Y_data[ids, 2] = ratings - m
            
            
    def loss(self):
        L = 0 
        for i in range(self.n_ratings):
            # user, item, rating
            n, m, rate = int(self.Y_data[i, 0]), int(self.Y_data[i, 1]), self.Y_data[i, 2]
            L += 0.5*(self.X[m, :].dot(self.W[:, n]) + self.b[m] + self.d[n] + self.mu - rate)**2
            
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro') + \
                          np.linalg.norm(self.b) + np.linalg.norm(self.d))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data[:,0] == user_id)[0] 
        item_ids = self.Y_data[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data[:,1] == item_id)[0] 
        user_ids = self.Y_data[ids, 0].astype(np.int32)
        ratings = self.Y_data[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            
            Wm = self.W[:, user_ids]
            dm = self.d[user_ids]
            xm = self.X[m, :]
            
            error = xm.dot(Wm) + self.b[m] + dm + self.mu - ratings 
            
            grad_xm = error.dot(Wm.T)/self.n_ratings + self.lam*xm
            grad_bm = np.sum(error)/self.n_ratings + self.lam*self.b[m]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
            self.b[m]    -= self.learning_rate*grad_bm
    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            bn = self.b[item_ids]
            wn = self.W[:, n]
            
            error = Xn.dot(wn) + bn + self.mu + self.d[n] - ratings
            grad_wn = Xn.T.dot(error)/self.n_ratings + self.lam*wn
            grad_dn = np.sum(error)/self.n_ratings + self.lam*self.d[n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
            self.d[n]    -= self.learning_rate*grad_dn
    
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw)
                print( 'iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        if self.user_based == 1:
            bias = self.muu[u]
        else:
            bias = self.muu[i]
        
        pred = self.X[i, :].dot(self.W[:, u]) + self.b[i] + self.d[u] + bias
#         if pred < 0:
#             return 0 
#         if pred > 5: 
#             return 5 
#         return pred 
        return max(0, min(5, pred))
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id])
        predicted_ratings= []
        for i in xrange(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
#             print pred, rate_test[n, 2]
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE
        
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.values#as_matrix()


rs = MF(Y_data, K = 3, max_iter = 1000, print_every = 100, lam = 0.1)

rs.fit()
rs.pred(6, 1)
print (rs.evaluate_RMSE(Y_data))

iter = 100 , loss = 0.4719197660655484 , RMSE train = 0.6352548018541067
iter = 200 , loss = 0.47522065722354623 , RMSE train = 0.6419490088984836
iter = 300 , loss = 0.4762308238824179 , RMSE train = 0.6438639553487919
iter = 400 , loss = 0.47649507882428555 , RMSE train = 0.6443633199902293
iter = 500 , loss = 0.4765650956741199 , RMSE train = 0.6444955539675876
iter = 600 , loss = 0.4765837346649721 , RMSE train = 0.6445307503731795
iter = 700 , loss = 0.47658870303706297 , RMSE train = 0.6445401318871977
iter = 800 , loss = 0.47659002786700827 , RMSE train = 0.64454263346722
iter = 900 , loss = 0.4765903811700975 , RMSE train = 0.6445433005818803
iter = 1000 , loss = 0.47659047539067256 , RMSE train = 0.6445434784911089
0.6445434784911089


# Áp dụng lên MovieLens 100k

In [4]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.values#as_matrix()
rate_test = ratings_test.values#as_matrix()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [5]:
rs = MF(rate_train, K = 100, lam = .01, print_every = 20, learning_rate = 2, max_iter = 200)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nUser-based MF, RMSE =', RMSE)

C:\Users\tvlon\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\tvlon\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iter = 20 , loss = 10.458239499760678 , RMSE train = 2.3109691008119744
iter = 40 , loss = 3.342884860424865 , RMSE train = 1.6924365911901758
iter = 60 , loss = 1.6481703982614002 , RMSE train = 1.2238439576448041
iter = 80 , loss = 1.0433441118168143 , RMSE train = 1.0441208667618564
iter = 100 , loss = 0.7419609668122962 , RMSE train = 0.9961510930658786
iter = 120 , loss = 0.5626056084434298 , RMSE train = 0.9842757721001738
iter = 140 , loss = 0.448195203429245 , RMSE train = 0.9811064669590487
iter = 160 , loss = 0.3735026429909729 , RMSE train = 0.9801689653847507
iter = 180 , loss = 0.3244220418721728 , RMSE train = 0.9798600316584866
iter = 200 , loss = 0.2921308434407418 , RMSE train = 0.9797459881734043

User-based MF, RMSE = 1.043729208439548


# MovieLens 1M

In [11]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')


C:\Users\tvlon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
from sklearn.model_selection import train_test_split
ratings = ratings_base.values#as_matrix()
ratings[:, :2] -= 1
rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print (rate_train.shape, rate_test.shape)

(670140, 4) (330069, 4)


In [13]:
rs = MF(rate_train, K = 2, lam = 0.1, print_every = 2, learning_rate = 2, 
        max_iter = 50, user_based = 0, Xinit = rs.X, Winit = rs.W)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

IndexError: index 5977 is out of bounds for axis 1 with size 943

In [14]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')

C:\Users\tvlon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
ratings = ratings_base.values#as_matrix()
ratings[:, :2] -= 1

from sklearn.model_selection import train_test_split

rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print (rate_train.shape, rate_test.shape)

rs = MF(rate_train, K = 2, lam = 0.1, print_every = 50, learning_rate = 10, max_iter = 500, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

(670140, 4) (330069, 4)
iter = 50 , loss = 0.24598298002066954 , RMSE train = 0.9731175621095624
iter = 100 , loss = 0.24598298002066954 , RMSE train = 0.9731175621095624
iter = 150 , loss = 0.24598298002066954 , RMSE train = 0.9731175621095624


KeyboardInterrupt: 